# Downloading AVHRR NDVI GIMMS3g


## Load packages

In [ ]:
import os
import ee
import wxee
import math
import eemont
import warnings
import xarray as xr
import pandas as pd
import geemap
import numpy as np
from odc.geo.xr import assign_crs

## Authenticate GEE    

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Analysis parameters


In [ ]:
# product = 'NOAA/CDR/AVHRR/SR/V5'
product = 'NASA/GIMMS/3GV0'
bands =  ['ndvi']
start_date = '1/1/1982'
end_date = '12/31/2013'
output_crs = "EPSG:3577"
resolution = 8000

# ~Aus region (slightly less to trick google into giving me 1km res)
region = ee.Geometry.Polygon([[
            [112,-43.0],
            [155,-43.0],
            [155,-9.0],
            [112,-9.0],
            [112,-43.0]]])


## QA mask

In [ ]:
def getQABits(image, start, end, newName):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start, end, 1):
       pattern += math.pow(2, i)

    out = image.select([0], [newName]) \
                  .bitwiseAnd(pattern) \
                  .rightShift(start)
    return(image.addBands(out))

def qa_gimms(image):
  good = getQABits(image.select("qa"),0,2,'good')
  good = good.lte(2)
  out = image.updateMask(good)
  return (out)

## Loop through months and export geotiff

In [ ]:
#use start and end dates to loop through months and load GEE data
start = pd.date_range(start=start_date, end=end_date, freq='MS') 
end = pd.date_range(start=start_date, end=end_date, freq='M')

i = 0
for s, e in zip(start,end):
    print(" {:03}/{:03}\r".format(i+1, len(start)), end="")

    ss = s+pd.Timedelta(14, 'd')

    s = s.strftime('%Y-%m-%d')
    e = e.strftime('%Y-%m-%d')

    ds = (ee.ImageCollection(product)
          .filterDate(s,e)
          .map(lambda image: image.clip(region))
          .map(lambda image: qa_gimms(image))
          .select(bands)
          .median()
          )

    export = ee.batch.Export.image.toDrive(ds,
        description='AVHRR_NDVI_GIMMS'+'_'+ss.strftime('%Y-%m-%d'),
        folder='AVHRR_NDVI_GIMMS',
        region=region,
        scale=resolution,
        crs=output_crs,
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        formatOptions={
            'cloudOptimized':True
              }
        )
    export.start()
    
    i+=1

In [ ]:
export.status()

## Plot on map

In [ ]:
# # Using basemap in geemap
# Map = geemap.Map(center=[-35.2041, 149.2721], zoom=6)

# ndviVis = {
#   'min': 0,
#   'max': 1.0,
#   'palette': ['000000', 'f5f5f5', '119701']}


# Map.addLayer(ds, ndviVis)
# Map

## Stitch together geotiffs

In [ ]:
import os
import xarray as xr
import numpy as np
import pandas as pd
from odc.geo.xr import assign_crs

import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _collect_prediction_data import round_coords

In [ ]:
base='/g/data/os22/chad_tmp/climate-carbon-interactions/data/NDVI_harmonization/AVHRR_NDVI_GIMMS/'
name= 'NDVI'
dates_start = '1982-01-01'
dates_end = '2013-12-31'


In [ ]:
files = [base+i for i in os.listdir(base) if i.endswith('.tif')]
files.sort()

time = (pd.date_range(dates_start, dates_end, freq='MS') + pd.DateOffset(days=14)).to_numpy()
time = list(time)

print(len(files), print(len(time)))

In [ ]:
i=0
arrs=[]
for f, t in zip(files, time):
    print(" {:02}/{:02}\r".format(i + 1, len(files)), end="")
    
    ds = xr.open_dataarray(f).squeeze().drop('band').astype('float32')
    ds = ds.where(ds>-1)
    ds = ds.expand_dims(time=[t])
    ds = ds.rename(name)
    del ds.attrs['long_name']
    del ds.attrs['AREA_OR_POINT']

    ds.attrs['nodata']=np.nan
    ds = ds.drop('spatial_ref')
    ds = assign_crs(ds, crs='epsg:3577')
    arrs.append(ds)
    i+=1 
    
ds = xr.concat(arrs, dim='time').sortby('time')
ds

In [ ]:
ds.sel(time='1994').plot.imshow(col='time', col_wrap=6, robust=True)

In [ ]:
ds.to_dataset().to_netcdf('/g/data/os22/chad_tmp/climate-carbon-interactions/data/NDVI_harmonization/AVHRR_GIMMS3g_'+name+'_5km_monthly_1982_2013.nc')